# Heuristics - CVRP

[OR-Tools Documentation](https://or-tools.github.io/docs/pdoc/ortools/constraint_solver/pywrapcp.html)

To do:

- Implement Engineer Input sheet (inc varying working hours)
- Use to calculate proper metrics
- Implement skills (and a couple more job types)
- Allow incomplete jobs (to mimic more jobs than time available!)

In [ ]:
from itertools import cycle

import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
import matplotlib as mpl
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

import functions as f
import importlib

## Instantiate data

In [ ]:
n_vehicles = 5
seniority_multiplier = [1.6, 1, 1.2, 1.6, 1.6]
start_locations = [4, 5, 6, 7, 10]
end_locations = [0, 0, 0, 0, 0] #all finish at dummy variable to allow arbitrary end locations

# assume indx 0 is the dummy end location
dataset = pd.read_csv("./data/custom_19.csv", index_col=0)
dataset_copy = dataset.copy()

for home_indx in start_locations:
    dataset.loc[home_indx, "timeslot"] = "AD"
    dataset.loc[home_indx, "job_type"] = "start"

coordinates = dataset.loc[:, ["x", "y"]]

N = coordinates.shape[0]

In [ ]:
# time windows, each each number represents 1 minute
time_window_dict = {
    "AD": [0, 510],  # 08:00 - 16:30
    "AM": [0, 240],  # 08:00 - 12:00
    "PM": [240, 510],  # 12:00 - 16:30
}

service_time_dict = {"start": 0, "DF-M Commision": 50, "DF-M Exchange": 100, "end": 0}

In [ ]:
time_matrix_multiplier = 1

# assume travel time is proportional to euclidean distance
time_matrix = squareform(pdist(coordinates, metric="euclidean"))

# reduce the drive times to make feasible solutions more likely, will adjust later!
time_matrix = np.round(time_matrix * time_matrix_multiplier, decimals=0).astype(int)

time_matrix[:, 0] = time_matrix[0, :] = (
    0  # the end location is a dummy variable used to allow arbitrary end locations
)

## Model

In [ ]:
# Create the routing index manager: number of nodes, number of vehicles, depot node
manager = pywrapcp.RoutingIndexManager(N, n_vehicles, start_locations, end_locations)

# Create Routing Model
routing = pywrapcp.RoutingModel(manager)

### Parameters

In [ ]:
# must create a different time_callback for each vehicle
def create_time_callback(vehicle_id):
    def time_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        travel_time = time_matrix[from_node, to_node]

        # convention is that job time added to drive FROM the job's node
        service_time = int(
            service_time_dict[dataset.loc[from_node, "job_type"]]
            * seniority_multiplier[vehicle_id]
        )

        return travel_time + service_time

    return time_callback


transit_callback_indices = []

# add callbacks so the solver can compute the time between nodes (including service times)
for vehicle_id in range(n_vehicles):
    transit_callback = create_time_callback(vehicle_id)
    transit_callback_index = routing.RegisterTransitCallback(transit_callback)
    transit_callback_indices.append(transit_callback_index)

### Constraints

In [ ]:
# Define time window contraints
time = "Time"

# create a CumulVar, for each location AND each vehicle
# 0 to N_locations-1 (index and node order is different!), N_locations to N_locations+N_vehicles-1
# VERY IMPORTANT, the use index for cumVars, not the node numbers
routing.AddDimensionWithVehicleTransits(
    transit_callback_indices,
    1000,  # allow waiting time
    1000,  # maximum time per vehicle (and location but the range is set later)
    True,  # start cumul to zero
    time,
)

time_dimension = routing.GetDimensionOrDie(time)

# add time window constraints for each location via the index!
# cannot do for the depot as is special case
for location_indx in dataset.index:
    if location_indx == 0:
        continue
    elif location_indx in start_locations:
        continue

    timeslot = dataset.loc[location_indx, "timeslot"]

    index = manager.NodeToIndex(location_indx)

    time_dimension.CumulVar(index).SetRange(
        time_window_dict[timeslot][0],
        time_window_dict[timeslot][1],  # window depends on promised timeslot
    )

# Add max drive time for each vehicle
# do this by setting the range of the end node for each vehicle
for vehicle_id in range(n_vehicles):

    end_index = routing.End(vehicle_id)
    time_dimension.CumulVar(end_index).SetRange(0, 510)  # 8.5 hours max

    # ensures engineer start during work hours, not strictly necessary but worth keeping in
    start_index = routing.Start(vehicle_id)
    time_dimension.CumulVar(start_index).SetRange(0, 510)  # 8.5 hours max

# this for loop encourage the optimizer to minimse the time of the last job of each vehicle
for i in range(n_vehicles):

    # start as early as possible
    routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.Start(i)))

    # finish as early as possible
    routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.End(i)))

### Objective

In [ ]:
# The cost of each arc is vehicle dependant!
# Must link to correct callback
for vehicle_id in range(n_vehicles):
    routing.SetArcCostEvaluatorOfVehicle(
        transit_callback_indices[vehicle_id], vehicle_id
    )

### Solution

In [ ]:
# Setting heuristic strategies
search_parameters = pywrapcp.DefaultRoutingSearchParameters()

search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
)

search_parameters.time_limit.FromSeconds(2)

# Solve the problem
solution = routing.SolveWithParameters(search_parameters)

In [ ]:
solution.ObjectiveValue()

### Plot Results

In [ ]:
calculated_objective_function_value = 0

cumulative_job_drive_time = 0
cumulative_drive_time = 0
cumulative_job_time = 0

print_val = "Key: |Node| A:Arrival, JT:Job Time, T:Travel Time, J:Job Time\n\n"
for vehicle_id in range(n_vehicles):
    index = routing.Start(vehicle_id)
    previous_index = None

    total_job_drive_time = 0
    total_drive_time = 0
    total_job_time = 0

    while not routing.IsEnd(index):

        previous_index = index
        previous_node = manager.IndexToNode(index)

        index = solution.Value(routing.NextVar(index))
        node = manager.IndexToNode(index)

        # arrival_time = solution.Min(time_dimension.CumulVar(previous_node))
        arrival_time = solution.Min(time_dimension.CumulVar(previous_index))

        job_drive_time = routing.GetArcCostForVehicle(previous_index, index, vehicle_id)

        total_job_drive_time += job_drive_time

        drive_time = time_matrix[previous_node][node]

        total_drive_time += drive_time

        job_time = service_time_dict[dataset.loc[previous_node, "job_type"]]

        total_job_time += job_time

        print_val += f"|{previous_node}| A:{arrival_time}, TS: {dataset.loc[previous_node, 'timeslot']}, JT:{job_drive_time}, T:{drive_time}, J:{job_time} -> "

    index = routing.End(vehicle_id)
    print_val += f"Finish: {solution.Value(time_dimension.CumulVar(index))} min\n"

    print_val += f"Totals: JT:{total_job_drive_time}, T:{total_drive_time}, J:{total_job_time}, Finish: {solution.Value(time_dimension.CumulVar(index))} min\n\n"

    cumulative_job_drive_time += total_job_drive_time
    cumulative_drive_time += total_drive_time
    cumulative_job_time += total_job_time

print_val += f"Totals: JT:{cumulative_job_drive_time}, T:{cumulative_drive_time}, J:{cumulative_job_time}\n"
print

print(print_val)

print(f"Objective function value: {solution.ObjectiveValue()} min")

print(f"Ob function directly minimises JT, not finish time")

In [ ]:
for location_indx in dataset.index:
    if location_indx == 0:
        continue
    timeslot = dataset.loc[location_indx, "timeslot"]
    time_var = time_dimension.CumulVar(manager.NodeToIndex(location_indx))
    print(
        f"Location {location_indx} ({timeslot}):  {solution.Value(time_var)}, {time_var}"
    )

print()
print("AM: [0, 240],  # 08:00 - 12:00 ")
print("PM: [240, 510],  # 12:00 - 16:30")

In [ ]:
importlib.reload(f)
tours = f.compile_tours(n_vehicles, routing, manager, solution, True)

In [ ]:
importlib.reload(f)
fig = f.plot_tours(tours, coordinates)
fig.show()